# Regression Analysis of Sabin data 

This analsys has been done in order to try and determine the context effects
of different context on the substitution probability
The input data is a csv files of the different K-mers and the $log[Pr(A\to B|S]$ of substitution rates of each of the passeges.

It includes 5 stages:

1. Feature extraction- creating a dummy variable for the categorical feature of context in a certain position
2. Feature selection- selecting the most important features using the randomized lasso regression method
3. Signifcance and effect test- after constructing the regression formula we are applying it in the regression to see if the effect of the motiff is significant also find the $\beta$ coeffs 

4. summarize the results across the different mutation types and passeges
5. Motif analysis - mining and displaying data about the different motifs to support decision about biological meaning of the motifs


In [1]:
%matplotlib
import pandas as pd
import itertools
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from treeinterpreter import treeinterpreter as ti
from scipy.stats import kendalltau,spearmanr
import statsmodels.formula.api as sm
import statsmodels.api as sma
from sklearn.linear_model import RandomizedLasso
import os
import glob
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
global MUTATIONS
MUTATIONS=['AC', 'GT', 'AG', 'CA', 'CG', 'GC', 'AT', 'GA', 'TG', 'CT', 'TC', 'TA']
global NUCS
NUCS=['A','C','G','T']
plt.style.use('ggplot')
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio.Seq import reverse_complement, transcribe, back_transcribe, translate
from sklearn.decomposition import PCA
from scipy.cluster.vq import whiten

from Bio import SeqIO
from scipy.stats import hypergeom


Using matplotlib backend: Qt4Agg


Sample of a data file in Passege 7

In [2]:
data=pd.read_csv(r'C:\Users\Guyling1\ContextProject\modeltest\nullwithP1P2GGCT.csv')
data.head(3)

,kmer,mutationType,P1,P2,P3,P4,P5,prob,indices
0,AACAA,AC,A,A,C,A,A,-20.723266,[]
1,AATAA,AT,A,A,T,A,A,-20.723266,[]
2,AAGAA,AG,A,A,G,A,A,-9.039006,"[2526, 3957, 4932, 6120, 6363, 6585, 6813]"


## 1.Feature Extraction

Defined a function that adds dummy variables of context dependence up to 3rd degree 


In [3]:
def generateFeature(k):
    for j in range(1,k+1):
        for feature in itertools.imap(''.join,itertools.combinations('1245',j)):
            yield feature

def functionGenerator(positions,x):
    res=''
    for position in positions:
        res+=x['P{}'.format(position)]
    return res

def createFeaturePrefix(featureList):
    res=''
    for feature in featureList:
        res+="P{}_".format(feature)
    return res

def getDummyVarPositionForK(data,k):
    for feature in generateFeature(k):
        featureList=[int(x) for x in list(feature)]
        concatFeature=data.apply(lambda x: functionGenerator(featureList,x),axis=1)
        dummies=pd.get_dummies(concatFeature,prefix=createFeaturePrefix(featureList))
        #print createFeaturePrefix(featureList) 
        data=pd.concat([data,dummies],axis=1)
    return data

def cleanDataMatrix(data,withNull=False):
    for i in range(1,6):
        data=data.drop('P{}'.format(i),axis=1)
    if not(withNull):
        data=data[data['prob']>-20]
    return data

Sample of the data after adding the dummy var features

In [4]:
data=getDummyVarPositionForK(data,3)
data=cleanDataMatrix(data)
data.head(3)

,kmer,mutationType,prob,indices,P1__A,P1__C,P1__G,P1__T,P2__A,P2__C,...,P2_P4_P5__TCG,P2_P4_P5__TCT,P2_P4_P5__TGA,P2_P4_P5__TGC,P2_P4_P5__TGG,P2_P4_P5__TGT,P2_P4_P5__TTA,P2_P4_P5__TTC,P2_P4_P5__TTG,P2_P4_P5__TTT
2,AAGAA,AG,-9.039006,"[2526, 3957, 4932, 6120, 6363, 6585, 6813]",1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AATAA,CT,-7.543296,"[1560, 1626, 3402, 4356, 6180, 6816, 6888, 7275]",1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,AACAA,TC,-8.986064,"[2919, 4191, 4449, 5394, 6381]",1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


After adding the dummy variables we want to create a feature matrix $X$ and predicted value vector $\bar{y}$  so that we can easily use it in the regression .

WithNull parameter chooses if we want to include unobserved mutations with their defult value of $10^{-9}$


In [ ]:

def createCovarsAndProbVectorForMutationType(data,mutType):
    data=data[data['mutationType']==mutType]
    probVector=data.prob
    covars=data.drop('prob',axis=1).drop('kmer',axis=1).drop('mutationType',axis=1).drop('indices',axis=1)
    return covars,probVector,data

def centerMatrix(data):
    return data-data.mean()

## 2.Feature Selection

Because we have a very large number of features and these features are also dependent (for example there is an obvious dependecy between P1=A and P1=A,P2=C) we use <a href="http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RandomizedLasso.html">Randomized lasso regression</a> which a randomized  $L-1$ norm based method of regression. We will only take in to account features that were used in over $ratio=0.2$ of the cases. This was done in order to solve problems with the forward feature selection methods that did not preform well

In [91]:
def featureToNumberOfOccurances(feature,data,allLen=False):
    places=pd.Series(data[data[feature]==1].index)
    if allLen:
        places=pd.Series(data.loc[(data[feature]==1) |(data[feature]==0)].index)
    return places.apply(lambda x: len(data.loc[x]['indices'])).sum()

def whitenMatrix(covars):
    index=covars.index
    columns=covars.columns
    pca = PCA(whiten=True)
    transformed=pca.fit_transform(covars)
    pca.whiten=False
    whiten=pca.inverse_transform(transformed)
    return pd.DataFrame(data=whiten,index=index,columns=columns)

def reduceEffectOfFeatures(mutData,features):
    newData=mutData.copy()
    print features
    probVector=mutData.prob
    predictors=" + ".join(features)
    result = sm.wls(formula="prob~ {}".format(predictors), data=mutData).fit()
    probVector=result.resid_pearson
    newData['prob']=probVector
    return newData
                        
def randomLassoFeatureExtraction(covars,probVector,mutData,ratio=0.2,partOfOccurances=0.01):
    featureList=[]
    total=featureToNumberOfOccurances('P1__A',mutData,allLen=True)
    #print total
    numOccurances=total*partOfOccurances
    names=covars.columns
    rlasso = RandomizedLasso(n_resampling=600)
    rlasso.fit(covars,probVector)
    sortedFeatures=sorted(zip(map(lambda x: round(x, 4), rlasso.scores_), 
                 names), reverse=True)
    for f in sortedFeatures:
        if (f[0]>ratio and featureToNumberOfOccurances(f[1],mutData)>numOccurances):
                featureList.append(f[1])
                #print featureToNumberOfOccurances(f[1],mutData),f[1]
        else:
            break
    return featureList

In [20]:
data

,kmer,mutationType,prob,indices,P1__A,P1__C,P1__G,P1__T,P2__A,P2__C,...,P2_P4_P5__TCG,P2_P4_P5__TCT,P2_P4_P5__TGA,P2_P4_P5__TGC,P2_P4_P5__TGG,P2_P4_P5__TGT,P2_P4_P5__TTA,P2_P4_P5__TTC,P2_P4_P5__TTG,P2_P4_P5__TTT
2,AAGAA,AG,-9.039006,"[2526, 3957, 4932, 6120, 6363, 6585, 6813]",1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AATAA,CT,-7.543296,"[1560, 1626, 3402, 4356, 6180, 6816, 6888, 7275]",1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,AACAA,TC,-8.986064,"[2919, 4191, 4449, 5394, 6381]",1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,AAAAA,GA,-10.582001,"[1953, 4077, 4080, 4140, 5226, 6360]",1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,AAGAC,AG,-9.350969,"[6051, 6480, 6819, 6918]",1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,AATAC,CT,-7.227841,"[1785, 1956, 3558, 4575, 5751, 6258, 7212]",1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,AACAC,TC,-10.056892,[2286],1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21,AAAAC,GA,-10.821671,"[936, 5148]",1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26,AAGAT,AG,-9.679961,"[2808, 3429, 3969, 4227, 5136, 6735, 6750, 7290]",1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28,AATAT,CT,-7.492476,"[963, 1239, 1746, 1893, 5079, 5244, 5274, 5478...",1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 3.Significance and effect test
Because every mutation context can only be compared to mutations of the same class (we might want to change this to transitions/transversions to get more power) the data is devided to different mutation Types and then we preform a regression when the covars are only features we obtained in the randomized lasso test.

In order to further decrease the effect of dependencies between the features I've used <a  href="http://statsmodels.sourceforge.net/devel/generated/statsmodels.regression.linear_model.WLS.html">WLS method</a>  instead of OLS which puts weights on the different expainatory variables based on the covariance matrix of the variables

In [234]:
def featureToMotiff(feature,mt,k=5,toString=True):
    kmer=['X' for i in range(k)]
    oldnuc=mt[0]
    kmer[k//2]=oldnuc
    positions=feature.split('__')[0]
    nucs=feature.split('__')[1]
    positions=[int(x[1]) for x in positions.split("_")]
    #print positions
    #print nucs
    for i in range(len(positions)):
        kmer[positions[i]-1]=nucs[i]
    if toString:
        return "".join(kmer)
    else:
        return kmer

def arrowBySign(float):
    if float>0:
        return "\u2191"
    else:
        return "\u2193"
    
def createNullFeatureList(nullDataPath):
    nullData=pd.read_csv(nullDataPath)
    nullData=getDummyVarPositionForK(nullData,3)
    nullData=cleanDataMatrix(nullData)
    return regressOverDataFile(nullData,justName=True,rawMotif=True,whiten=False)
    

def regressOverDataFile(data,pvalCutOff=10**-5,justName=False,rawMotif=False,whiten=False,nullDataPath=None):
    mutationTypes=set(data['mutationType'].values)
    #print mutationTypes
    motifMap={}
    if nullDataPath:
        nullMap=createNullFeatureList(nullDataPath)
        #print nullMap
    for mt in mutationTypes:
        #print mt
        motifMap[mt]=[]
        covars,probVector,mutData=createCovarsAndProbVectorForMutationType(data,mt)
        if whiten:
            try:
                nullFeatures=nullMap[mt]
            except:
                return []
        covars=centerMatrix(covars)
        covars=whitenMatrix(covars)
        features=randomLassoFeatureExtraction(covars,probVector,mutData)
        if len(features)==0:
            #print "no significant features"
            continue
        if whiten:
            if len(nullFeatures)>0:
                #print "null features: "+str(nullFeatures)
                #mutData=reduceEffectOfFeatures(mutData,nullFeatures)
                features=list(set(features)-set(nullFeatures))
        predictors=" * ".join(features)
        #print predictors
        result = sm.wls(formula="prob~ {} +1".format(predictors), data=mutData).fit()
        sigFeatures=result.pvalues.sort_values()[1:20].keys()
        #print("model r^2 {}".format(result.rsquared_adj))
        for f in sigFeatures:
            if result.pvalues[f]<pvalCutOff:# or not(whiten):
                if rawMotif:
                    if justName:
                        motifMap[mt].append(f)#,result.pvalues[f],result.params[f],result.conf_int().loc[f][0],result.conf_int().loc[f][1])
                    else:
                        motifMap[mt].append((f,result.pvalues[f],result.params[f],result.conf_int().loc[f][0]\
                                             ,result.conf_int().loc[f][1]))
                else: 
                    if justName:
                        motifMap[mt].append((featureToMotiff(f,mt)+arrowBySign(result.params[f]).decode('unicode-escape')))#,result.pvalues[f],result.params[f],result.conf_int().loc[f][0],result.conf_int().loc[f][1])
                    else:
                        motifMap[mt].append((featureToMotiff(f,mt),result.pvalues[f],result.params[f],result.conf_int().loc[f][0]\
                                             ,result.conf_int().loc[f][1]))
    return motifMap

In [184]:
data=pd.read_csv(r'C:\Users\Guyling1\ContextProject\modeltest\nullwithP1P2GGCT.csv')
data=getDummyVarPositionForK(data,3)
data=cleanDataMatrix(data)

motiffMap=regressOverDataFile(data,rawMotif=True,justName=False\
,whiten=True,nullDataPath=r'C:\Users\Guyling1\ContextProject\modeltest\null.csv')
motiffMap

{'AC': [],
 'AG': [],
 'AT': [],
 'CA': [],
 'CG': [],
 'CT': [],
 'GA': [],
 'GC': [],
 'GT': [],
 'TA': [],
 'TC': [],
 'TG': []}

## 4.Summarize different passeges and mutation types 
I created another function to give us the effect across diffetent passeges

In [230]:
def fileToPassege(fileName):
    return fileName.split(".")[0][-2:]

def regressOverFolder(folder,justNameVar=True,rawMotif=False):
    os.chdir(folder)
    files=glob.glob("*.csv")
    passeges=[fileToPassege(fileName) for fileName in files]
    motifTable=pd.DataFrame('',index=passeges,columns=MUTATIONS)
    #print motifTable
    for filee in files:
        data=pd.read_csv(filee)
        #print filee
        data=getDummyVarPositionForK(data,3)
        data=cleanDataMatrix(data)
        fileMotifMap=regressOverDataFile(data,justName=justNameVar,rawMotif=rawMotif)#\
                                         #,nullDataPath=folder+"null\{}".format(filee.split(".")[0]+"."+filee.split(".")[-1]),whiten=True)
        for mut in fileMotifMap.keys():
            if len(fileMotifMap[mut])>0:
                motifTable.loc[fileToPassege(filee)][mut]=fileMotifMap[mut]
    return motifTable

<h2> Sabin Analysis </h2>
<h3> regular sample </h3>

In [211]:
regressOverFolder(r'C:\Users\Guyling1\ContextProject\SABIN',rawMotif=False)

regressionCSVOneFileK5P1.csv
regressionCSVOneFileK5P2.csv
regressionCSVOneFileK5P3.csv
regressionCSVOneFileK5P4.csv
regressionCSVOneFileK5P5.csv
regressionCSVOneFileK5P6.csv
regressionCSVOneFileK5P7.csv


,AC,GT,AG,CA,CG,GC,AT,GA,TG,CT,TC,TA
P1,[XCAXX↑],,,,,,,,,[XXCCX↓],[XXTAA↑],
P2,[XCAXX↑],,[XGAXX↓],[CXCCA↓],,,,,,,[XXTAX↑],
P3,[XCAXX↑],,,,,,[XXATG↓],,,[XXCCX↓],[XXTAA↑],
P4,[XCAXX↑],,,,,,,,,,[XXTAX↑],
P5,[XCAXX↑],,[CAATX↓],,,,,[XGGXC↑],,[XXCCX↓],[XXTAA↑],
P6,,,,,,,,,,[XXCCX↓],[XXTAA↑],
P7,[AXATA↑],,,,,,,,,[XXCCX↓],,


In [ ]:
#rawMotifSummarySABIN=regressOverFolder(r'C:\Users\Guyling1\ContextProject\SABIN4Fold',justNameVar=False,rawMotif=False)
rawMotifSummarySABIN['TC']['P1']

<h3> Only 4-fold degenerate sites</h3>

In [213]:
regressOverFolder(r'C:\Users\Guyling1\ContextProject\SABIN4Fold',rawMotif=False)

regressionCSVOneFileK5P1.csv
regressionCSVOneFileK5P2.csv
regressionCSVOneFileK5P3.csv
regressionCSVOneFileK5P4.csv
regressionCSVOneFileK5P5.csv
regressionCSVOneFileK5P6.csv
regressionCSVOneFileK5P7.csv


,AC,GT,AG,CA,CG,GC,AT,GA,TG,CT,TC,TA
P1,,,,,,,,,[XXTGX↓],,,[XGTTX↑]
P2,[XCAXX↑],,,,,,,[GTGXX↓],"[XXTTX↑, XTTAG↑]",[XXCGX↑],,
P3,[XCAXX↑],"[GXGGG↓, XXGAX↓]",,,[XGCXG↓],,,,,[XXCGX↑],,
P4,[XCAXX↑],,,,,,,,[XXTTX↑],[XXCGX↑],,
P5,[XTAXX↓],,,,,,,,,,[XXTAX↑],
P6,,,[XTACC↑],,,,[XTAXX↓],[XXGCX↑],[XXTGX↓],,,
P7,,,,,,,,,,,,


In [214]:
rawMotifSummarySABIN4Fold=regressOverFolder(r'C:\Users\Guyling1\ContextProject\SABIN4Fold',justNameVar=False,rawMotif=True)
rawMotifSummarySABIN4Fold.loc['P5']['AC']

regressionCSVOneFileK5P1.csv
regressionCSVOneFileK5P2.csv
regressionCSVOneFileK5P3.csv
regressionCSVOneFileK5P4.csv
regressionCSVOneFileK5P5.csv
regressionCSVOneFileK5P6.csv
regressionCSVOneFileK5P7.csv


[('P2__T',
  5.1166970087139269e-11,
  -1.2738445365645075,
  -1.6162220420032734,
  -0.93146703112574147)]

<h3> scrambled codons</h3>


In [235]:
regressOverFolder(r'C:\Users\Guyling1\ContextProject\SABINscrambled',rawMotif=False)

,AC,GT,AG,CA,CG,GC,AT,GA,TG,CT,TC,TA
P1,,,,,,,[XTAGX↑],,,,,
P2,,,,,[TXCGX↑],,[XTAGX↑],,,,,
P3,,,[XXAGX↑],,,,,,,,,
P4,,,,,,,,,,,,
P5,,,,,,,,,,,,
P6,,,,,,,,[AAGAX↑],,,,
P7,,,,,,,,,,,,


In [236]:
rawMotifSummarySABINScrambled=regressOverFolder(r'C:\Users\Guyling1\ContextProject\SABINscrambled',justNameVar=False,rawMotif=True)
rawMotifSummarySABINScrambled.loc['P4']['TC']

''

<h2>SD analysis</h2>
<h3> regular sample </h3>


In [224]:
regressOverFolder(r'C:\Users\Guyling1\ContextProject\SD',justNameVar=True,rawMotif=False)

,AC,GT,AG,CA,CG,GC,AT,GA,TG,CT,TC,TA
P1,,,,,[CXCGX↑],,,,,[XACAG↑],[XXTCX↓],
P2,,,,,,,,,,,[XXTAX↑],
P3,,,,,[CXCGX↑],,,,[XGTXX↑],,[XXTAX↑],
P4,,,[XXAGX↑],,[CXCGX↑],,,[XCGXX↑],[CGTGX↑],"[CACAX↑, XXCCX↓]",[XXTAX↑],
P5,,,,,,,,[XCGXX↑],,,[XXTAX↑],
P6,,,[GTAXX↑],,[CXCGX↑],,,[XCGXX↑],,,[XXTAX↑],
P7,,,,,,,[GXAXX↑],[XCGXX↑],[CGTGX↑],,[XXTAX↑],


In [225]:
#rawMotifSummarySD=regressOverFolder(r'C:\Users\Guyling1\ContextProject\SD',justNameVar=False,rawMotif=True)
rawMotifSummarySD.loc['P6']['CG']

[('P1_P4__CG',
  2.0911829518537129e-05,
  2.1248303269263702,
  1.2076911355587672,
  3.041969518293973)]

<h3> Only 4-fold degenerate sites</h3>

In [226]:
regressOverFolder(r'C:\Users\Guyling1\ContextProject\SD4Fold',justNameVar=True,rawMotif=False)

,AC,GT,AG,CA,CG,GC,AT,GA,TG,CT,TC,TA
P1,,,[XTAXX↑],,[CXCGX↑],,,,,,,
P2,,,[GTAXX↑],,[CXCGG↑],,,,[XGTXX↑],[XGCCA↑],,
P3,,,[GTAXX↑],,,,[GGAXC↑],,,,[XXTAX↑],
P4,,,[GTAXX↑],,,,,,[CGTGX↑],,[XXTAX↑],
P5,,,[GTAXX↑],[XTCAG↓],,,[XGAXC↑],,,,[XXTAX↑],
P6,,,[GTAXX↑],,,,,,,[XGCCA↑],"[XXTAX↑, XXTXG↓]",
P7,,,[GTAXX↑],,,,[CXAXX↓],,,,[XXTAX↑],[GXTCA↑]


In [227]:
#rawMotifSummarySD4Fold=regressOverFolder(r'C:\Users\Guyling1\ContextProject\SD4Fold',justNameVar=False,rawMotif=True)
rawMotifSummarySD4Fold.loc['P6']['AC']

''

<h3> scrambled codons</h3>


In [62]:
regressOverFolder(r'C:\Users\Guyling1\ContextProject\SDscrambled',justNameVar=True,rawMotif=False)

,AC,GT,AG,CA,CG,GC,AT,GA,TG,CT,TC,TA
P1,,,,[XXCGX↑],,,,,,,,
P2,[XAAGT↑],[XCGTX↑],,,,,,,,,,
P3,,[XCGTX↑],,,,,,,,,,
P4,,,[XGACX↓],,"[XXCGC↑, CXCXA↑, XGCTX↑]",,[XCAXC↑],,,[XXCGX↑],,
P5,,[XCGTX↑],[XGACX↓],,,,,"[XXGAX↓, AXGAC↓]",,,,
P6,,[XCGTX↑],,,,,,[CAGAX↓],,,,
P7,,,[XGACX↓],,,,,"[XXGAX↓, CXGAT↓, CAGAX↓]",,,,


In [21]:
#rawMotifSummarySDscrambled=regressOverFolder(r'C:\Users\Guyling1\ContextProject\SDscrambled',justNameVar=False,rawMotif=True)
rawMotifSummarySDscrambled.loc['P4']['TC']

''

In [238]:

#rawMotifSummary=regressOverFolder(r'C:\Users\Guyling1\ContextProject\SABINnull',justNameVar=False,rawMotif=True)
#regressOverFolder(r'C:\Users\Guyling1\ContextProject\SABINnull',rawMotif=False)

In [239]:
#regressOverFolder(r'C:\Users\Guyling1\ContextProject\SABIN4FoldScrambled',rawMotif=False)

In [240]:
#rawMotifSummary.loc['P4']['TA']

## 5.Motif Analsys

### Comparing Histograms of sequences with the motif and sequences without the motif

In [45]:
def seperateMotifAndNonMotif(motif,data,mutationType):
    data=data[data['mutationType']==mutationType]
    nonMotifData=data[data[motif]==0]
    motifData=data[data[motif]==1]
    print motifData.prob
    return motifData,nonMotifData,motif

def histOfMotfAndNonMotif(motif,nonMotif,motifName):
    motifProb=pd.Series(motif.prob)
    nonMotifProb=pd.Series(nonMotif.prob)
    df=pd.DataFrame({'Motif':motifProb,'nonMotif':nonMotifProb},columns=['Motif','nonMotif'])
    plt.figure()
    motifProb.plot.kde()
    nonMotifProb.plot.kde()
    plt.title("{} motif analysis".format(motifName))
    print motifProb.mean(),nonMotifProb.mean()
    plt.text(-11,0.4,"motif mean=%.2f" % motifProb.mean())
    plt.text (-11,0.3,"nonMotif mean=%.2f" % nonMotifProb.mean())

In [47]:
motif,nonMotif,motifName=seperateMotifAndNonMotif('P4__C',data,'TC')
histOfMotfAndNonMotif(motif,nonMotif,motifName)

55     -12.287802
67      -8.428613
79      -8.543566
247     -9.690290
259    -10.253223
271    -10.304970
439    -10.051629
451     -9.239390
463     -9.891528
631     -8.796771
643     -8.722543
655    -11.394609
667     -8.908196
823    -10.650806
835     -8.679025
1027   -10.464850
1591    -9.662945
1603    -9.900120
1615   -10.770827
1783    -9.851998
1795    -7.910850
1807   -10.372709
1975   -10.407042
1987    -9.854987
1999   -10.078869
2167   -10.143964
2179    -9.355371
2191    -9.563512
2359    -9.282508
2371    -8.198912
2383    -9.125827
2551   -10.057641
2563    -8.479803
2575    -9.652295
2587    -9.566557
2743   -10.855034
2755    -8.337121
2779   -10.031684
2935    -9.962288
2947    -9.467642
Name: prob, dtype: float64
-9.67995790556 -9.40144567744


More data about the motifs that could give an idea about other factors influencing the substitution rates other than the motifs

In [456]:

def parseSeq(seqLocation,codingStart,codingEnd):
    fasta_sequences = SeqIO.parse(open(seqLocation),'fasta')
    for fasta in fasta_sequences:
        consensus= fasta.seq[codingStart:codingEnd]
        return consensus

def isSynMutation(seq,kmer,offset,k=5):
    originalSeq=str(seq)
    mutatedSeq=str(seq)  
    mutatedSeqList=list(mutatedSeq)
    mutatedSeqList[offset-k//2:offset+k//2+1]=list(kmer)
    mutatedSeq="".join(mutatedSeqList)
    return translate(originalSeq)==translate(mutatedSeq)
    
def checkMotiflocations(seq,motif,mutationType):
    locations=[]
    motif=featureToMotiff(motif,mutationType,toString=False)
    k=len(motif)
    #print motif
    allIndices=set([i for i in range(k)])
    freeIndices=set([i for i, x in enumerate(motif) if x == "X"])
    #print freeIndices
    checkIndices=list(allIndices-freeIndices)
    #print checkIndices
    for i in range(k//2,len(seq)-k//2):
        isMotif=True
        compared=list(seq[i-k//2:i+k//2+1])
        #print compared
        for position in checkIndices:
            #print (compared[position],motif[position])
            if compared[position]!=motif[position]:
                isMotif=False
                break
        comparedAfterMutation=compared
        comparedAfterMutation[k//2]=mutationType[1]
        if (isMotif and isSynMutation(seq,comparedAfterMutation,i)):
            locations.append(i)
    return locations

def is4FoldDegenerate(seq,location,k=5):
    compared=list(seq[location-k//2:location+k//2+1])
    for nuc in NUCS:
        compared[k//2]=nuc
        if not(isSynMutation(seq,compared,location)):
            return False
    return True


def iterateOverLocationsList(function,seq,locations,k=5):
    resList=[]
    for location in locations:
        if function(seq,location):
            resList.append(location)
    return resList
            
def hyperGeoTest(M,n,locations,TrueLocations):
    rv = hypergeom(M, n, locations)
    moreThanTrue=[rv.pmf(TrueLocation) for TrueLocation in range(TrueLocations,locations+1)]
    #print moreThanTrue
    return sum(moreThanTrue)

def checkHyperGeometricForAllMotifs(motifTable,function,M,n,seq,thrashold=0.05):
    sigHG={}
    mutations=motifTable.columns
    for mt in mutations:
        lenn=len(motifTable[mt])
        tableByMt=motifTable[mt]
        for i in range(lenn):
            cellMotifs=tableByMt[i]
            if cellMotifs!="":
                for motif in cellMotifs:
                    #print motif
                    allLocations=checkMotiflocations(seq,motif,mt)
                    #print len(allLocations)
                    trueLocations=iterateOverLocationsList(function,seq,allLocations)
                    #print len(trueLocations)
                    pvalue=hyperGeoTest(M,n,len(allLocations),len(trueLocations))
                    #print pvalue
                    if (featureToMotiff(motif,mt),mt) in sigHG.keys():
                        continue
                    if (pvalue < thrashold):
                        sigHG[(featureToMotiff(motif,mt),mt)]=pvalue
    return pd.DataFrame(sigHG.items(),columns=['motif_mutationType','HG test PV'])

<h3>4-fold Degenerate sites enrichment</h3>

We want to check the if there is any kind of enrichment for 4-fold degenerate site. in order to do this we will preform a HG test when $n=994$ is the number of 4-fold degenerate sites within the coding region and $M=2345$ possible sites of synonymous mutation.

<b>There appears to be an significant large number of 4-fold degenerate sites in many of the motif we found as having an effect on mutation rate</b>

In [404]:
cons=parseSeq(r'C:\Users\Guyling1\ContextProject\AggarwalaPaper\sabin2.full_genome.U882C.A2973G.C4905U.C5526U.fasta',747,7638)
checkHyperGeometricForAllMotifs(rawMotifSummary,is4FoldDegenerate,2345,994,cons)

,motif_mutationType,HG test PV
0,"(GXTTG, TA)",3.217076e-02
1,"(AXATA, AC)",3.217076e-02
2,"(XXTAX, TC)",3.768880e-02
3,"(GCCXX, CG)",7.755594e-16
4,"(XCAXX, AC)",7.248911e-53
5,"(XXTCC, TG)",1.362744e-05
6,"(XCTXG, TA)",1.220260e-11


In [237]:

def createLinePlotByMTForFolder(folder):
    os.chdir(folder)
    files=glob.glob("*.csv")
    firstTable=pd.read_csv(files[0])
    mutTuple=pd.Series(firstTable.index).apply(lambda x: firstTable.loc[x]['kmer']+" "+firstTable.loc[x]['mutationType'])
    passeges=[fileToPassege(fileName) for fileName in files]
    sumTable=pd.DataFrame('',index=mutTuple,columns=passeges)
    for f in files:
        name=fileToPassege(f)
        tempTable=pd.read_csv(f)
        sumTable[name]=tempTable['prob'].values
    plt.figure()
    sumTable.transpose().to_csv(folder+"\summary.csv")
    

    
        
        

#createLinePlotByMTForFolder(r'C:\Users\Guyling1\ContextProject\refactoredCode')
        
        
        
        
    
    